# Unified Inference System

> Intelligent inference system that automatically selects between serial, parallel, and HPC multinode execution


In [2]:
#| default_exp inference.unified_inference


In [3]:
#| hide
%load_ext autoreload
%autoreload 2


## Overview

This notebook provides a unified inference system that intelligently chooses between three execution strategies:

1. **Jupyter Notebook Mode**: Simple for-loop execution for interactive development
2. **HPC Multinode Mode**: Distributed execution across cluster nodes using `bsub`
3. **Parallel Mode**: Local parallel execution using multiprocessing.Pool

The system automatically detects the execution environment and selects the optimal strategy.


## Imports


In [4]:
#| export
import os
import sys
import shutil
import numpy as np
import subprocess
from pathlib import Path
from typing import Union, List, Dict, Any, Optional, Set,Callable
from multiprocessing import Pool, cpu_count
import json
from tqdm import tqdm
import os
from platform import system

from fastcore.all import *
from fastcore.test import *


In [5]:
from nbdev.showdoc import *


In [6]:
if system() == 'Linux':
    os.chdir(r'/home/ai_dsx.work/data/projects/be-vision-ad-tools')

In [7]:
#| export
# Import from existing notebooks
from be_vision_ad_tools.inference.prediction_system import (
    predict_image_list_from_file_enhanced
)

from be_vision_ad_tools.inference.multinode_inference import *

from be_vision_ad_tools.inference.multinode_from_aiop_tool import (
    HPC_Job,
    DistributeHPC,
    print_status
)


# Data

In [8]:
DATA_CORE = "/home/ai_dsx.work/data/projects/AD_tool_test/images/"

MODEL_PATH= Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/exports/TEST_MULITNODE_task_000_padim_resnet18_18_layer1/weights/torch/model.pt')
GOOD_IM_PATH= Path(DATA_CORE, 'good')
BAD_IM_PATH= Path(DATA_CORE, 'bad')
OUTPUT_DIR = Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009')
good_im_list = GOOD_IM_PATH.ls()
bad_im_list = BAD_IM_PATH.ls()


## Smart Folder Handling

Functions to handle both flat and nested folder structures.


In [17]:
#| export
def resolve_test_folders_smart(
    test_folders: Union[str, Path, List[Union[str, Path]]]  # Folder(s), file(s), or mixed
) -> List[Path]:  # Returns list of image paths
    """Resolve test_folders to image paths - handles lists, flat folders, and nested folders."""
    
    if not isinstance(test_folders, list):
        test_folders = [test_folders]
    
    image_paths = []
    
    for folder_or_file in test_folders:
        path = Path(folder_or_file)
        
        if not path.exists():
            print(f"⚠️  Warning: '{folder_or_file}' does not exist")
            continue
        
        if path.is_file() and is_image_file(path):
            # It's an image file
            image_paths.append(path)
            
        elif path.is_dir():
            # It's a directory - use smart folder scanning
            try:
                folder_info = scan_folder_structure(path)
                image_paths.extend(folder_info['all_image_paths'])
            except Exception as e:
                print(f"⚠️  Warning: Failed to scan '{folder_or_file}': {e}")
                # Fallback to old behavior
                for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.tif']:
                    image_paths.extend(path.glob(ext))
                    image_paths.extend(path.glob(ext.upper()))
        else:
            print(f"⚠️  Warning: '{folder_or_file}' is not a valid file or directory")
    
    # Remove duplicates and sort
    unique_paths = sorted(set(image_paths))
    
    print(f"📁 Resolved {len(unique_paths)} images from {len(test_folders)} input path(s)")
    return unique_paths


In [18]:
from nbdev.showdoc import *

In [19]:
doc(resolve_test_folders_smart)

,Type,Details,
test_folders,Union,"Folder(s), file(s), or mixed",
Returns,List,Returns list of image paths,


In [18]:
all_images = resolve_test_folders_smart(
    test_folders=list(good_im_list),

)

📁 Resolved 85 images from 85 input path(s)


In [19]:
all_images_ = resolve_test_folders_smart(
    test_folders=DATA_CORE

)

📁 Detected NESTED structure in: /home/ai_dsx.work/data/projects/AD_tool_test/images
   📂 Lot 'good': 85 images
   📂 Lot 'bad': 2 images
   📂 Lot 'hyperparameter_models': 1 images

✅ Scan complete: 88 total images
📁 Resolved 88 images from 1 input path(s)


## Environment Detection

These functions detect the execution environment to choose the optimal inference strategy.


In [20]:
#| export
def in_jupyter_notebook() -> bool:
    """Check if code is running in a Jupyter notebook."""
    try:
        # Check for IPython shell
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True  # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type
    except NameError:
        return False  # Probably standard Python interpreter


In [21]:
in_jupyter_notebook()

True

In [22]:
#| export
def has_bsub_command() -> bool:
    """Check if bsub command is available (HPC environment)."""
    return shutil.which("bsub") is not None


In [23]:
has_bsub_command()

True

In [24]:
#| export
def detect_execution_environment() -> str:
    """Detect execution environment and return appropriate mode."""
    if in_jupyter_notebook():
        return "jupyter"
    elif has_bsub_command():
        return "hpc"
    else:
        return "parallel"


In [25]:
detect_execution_environment()

'jupyter'

In [26]:
# Test environment detection
def test_environment_detection():
    """Test environment detection functions."""
    
    # Test Jupyter detection
    jupyter_result = in_jupyter_notebook()
    print(f"📓 In Jupyter: {jupyter_result}")
    
    # Test HPC detection
    hpc_result = has_bsub_command()
    print(f"🖥️  Has bsub: {hpc_result}")
    
    # Test overall detection
    mode = detect_execution_environment()
    print(f"🎯 Detected mode: {mode}")
    
    assert mode in ["jupyter", "hpc", "parallel"], f"Invalid mode: {mode}"
    print("✅ Environment detection tests passed")

test_environment_detection()


📓 In Jupyter: True
🖥️  Has bsub: True
🎯 Detected mode: jupyter
✅ Environment detection tests passed


## Execution Strategies

Three different execution strategies for different environments.


### Strategy 1: Serial Execution (Jupyter)


In [46]:
doc(predict_image_list_from_file_enhanced)

,Type,Default,Details,
model_path,Union,,"path to the model(.ckpt, .pt, .onnx, .xml)",
image_list_file,Union,,text file with one image path per line,
batch_id,Optional,None,unique identifier for this batch (for parallel processing),
output_dir,Union,None,directory to save the heatmap,
save_heatmap,bool,False,whether to save heatmap visualizations,
heatmap_style,str,side_by_side,"""heatmap_only"", ""combined"", ""side_by_side"", ""cv2_side_by_side"", ""cv2_heatmap_only""",
device,str,auto,"device to use for prediction(""auto"", ""cpu"", ""cuda"")",
save_results,bool,True,whether to save JSON results,
show_heatmap,bool,False,Whether to show heatmap,
compress,bool,True,Whether to compress the image (JPEG format),


In [ ]:
#| export
def run_jupyter_inference(
    model_path: Union[str, Path], # Path to the model file
    image_path: Union[str, Path, List[Path]], # List of image paths to process
    output_dir: Union[str, Path] = "inference_results", # Output directory
    save_heatmaps: bool = False, # Whether to save heatmaps
    heatmap_style: str = "side_by_side", # Style of heatmap to save
    jpeg_quality: int = 95,
    compress: bool = True,
    preprocessing_fn=None, # Preprocessing function
    preprocessing_kwargs=None, # Preprocessing kwargs
    **kwargs
) -> Dict[str, Any]:
    """Execute inference serially using simple for-loop (Jupyter mode)."""

    image_paths = resolve_test_folders_smart(image_path)
    
    print(f"📓 Running in Jupyter mode (serial for-loop)")
    print(f"   Processing {len(image_paths)} images sequentially...")
    
    model_path = Path(model_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create temporary batch file with all images
    batch_file = output_dir / "jupyter_batch_images.txt"
    create_batch_list_file(image_paths, batch_file)
    
    # Use existing prediction system
    results = predict_image_list_from_file_enhanced(
        model_path=model_path,
        image_list_file=batch_file,
        batch_id="jupyter_batch",
        output_dir=output_dir,
        save_results=save_heatmaps,
        heatmap_style=heatmap_style,
        compress=compress,
        jpeg_quality=jpeg_quality,
        preprocessing_fn=preprocessing_fn,
        preprocessing_kwargs=preprocessing_kwargs,
        **kwargs
    )
    
    print(f"✅ Jupyter inference complete: {len(image_paths)} images processed")
    
    return {
        "mode": "jupyter",
        "total_images": len(image_paths),
        "output_dir": str(output_dir),
        "results": results
    }


In [28]:
from nbdev.showdoc import *

In [29]:
from be_vision_ad_tools.inference.prediction_system import (
    predict_image_list_from_file_enhanced,
    predict_image_list)

In [30]:
def create_half_black_image(
    image: np.ndarray, # Input image as numpy array
    side: str = "left" # Side to make black: "left", "right", "top", or "bottom"
) -> np.ndarray:
    """Create an image with half of it blacked out."""
    if not isinstance(image, np.ndarray):
        raise TypeError("image must be a numpy array")
    
    if side not in ["left", "right", "top", "bottom"]:
        raise ValueError(f"Invalid side: {side}. Must be one of: left, right, top, bottom")
    
    # Create a copy to avoid modifying the original
    result = image.copy()
    
    height, width = image.shape[:2]
    
    if side == "left":
        result[:, :width//2] = 0
    elif side == "right":
        result[:, width//2:] = 0
    elif side == "top":
        result[:height//2, :] = 0
    elif side == "bottom":
        result[height//2:, :] = 0
    
    return result

In [48]:
rs = run_jupyter_inference(
    model_path=MODEL_PATH,
    image_paths=bad_im_list,
    output_dir=OUTPUT_DIR,
    save_heatmaps=True,
    heatmap_style="side_by_side",
    #preprocessing_fn=create_half_black_image,
    #preprocessing_kwargs={"side": "left"}
    jpeg_quality=95,
    compress=True,
)

2025-11-07 18:59:20,908 - be_vision_ad_tools.inference.prediction_system - INFO - Predicting with .pt model on 3042400443552714.png


📓 Running in Jupyter mode (serial for-loop)
   Processing 2 images sequentially...
🚀 ENHANCED PREDICT IMAGE LIST FROM FILE
🎯 Reading image list from: /home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009/jupyter_batch_images.txt
📂 Loaded 2 image paths
🎨 Style: side_by_side
🔍 Processing 2 images from list
📦 Batch ID: jupyter_batch
🎯 Using model: model.pt


2025-11-07 18:59:21,297 - be_vision_ad_tools.inference.prediction_system - INFO - Prediction: NORMAL (Score: 0.2050)
2025-11-07 18:59:21,298 - be_vision_ad_tools.inference.prediction_system - INFO - Predicting with .pt model on 3042400444552714.png
2025-11-07 18:59:21,674 - be_vision_ad_tools.inference.prediction_system - INFO - Prediction: ANOMALY (Score: 1.0000)



📊 Batch Processing Summary:
   Batch ID: jupyter_batch
   Images in List: 2
   Valid Images: 2
   Successfully Processed: 2
   Failed: 0
   Normal: 1 (50.0%)
   Anomalies: 1 (50.0%)
   Average Score: 0.6025
Saving results to JSON, output directory: /home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009/batch_jupyter_batch
💾 Results saved to: /home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009/batch_jupyter_batch/batch_results_jupyter_batch.json
✅ Jupyter inference complete: 2 images processed


In [51]:
rs['results'].keys()

dict_keys(['statistics', 'results'])

## Strategy 2: HPC Multinode Execution


In [53]:
def run_hpc_inference(
    model_path: Union[str, Path],
    image_path: Union[str, Path, List[Path]],
    output_dir: Union[str, Path] = "inference_results",
    save_heatmaps: bool = False,
    heatmap_style: str = "side_by_side",
    compress: bool = True,
    jpeg_quality: int = 95,
    batch_size: int = 10,
    num_nodes: int = 10,
    preprocessing_fn=None,
    preprocessing_kwargs=None,
    **kwargs) -> Dict[str, Any]:
    """Execute inference using HPC multinode execution."""
    
    print(f"🖥️  Running HPC multinode inference")
    bsub_rs = distribute_folder_inference(
                                        root_path=image_path,
                                        model_path=model_path,
                                        output_dir=output_dir,
                                        save_heatmaps=save_heatmaps,
                                        heatmap_style=heatmap_style,
                                        batch_size=batch_size,
                                        num_nodes=num_nodes,
                                        compress=compress,
                                        jpeg_quality=jpeg_quality,
                                        preprocessing_fn=preprocessing_fn,
                                        preprocessing_kwargs=preprocessing_kwargs,
                                        **kwargs
                                    )

    print(f"✅ HPC inference complete: {len(Path(image_path).ls())} images processed")
    
    return {
        "mode": "hpc",
        "total_images": len(Path(image_path).ls()),
        "output_dir": str(output_dir),
        "num_nodes": num_nodes,
        "results": bsub_rs
    }
    

    

    

In [54]:
OUTPUT_DIR

Path('/home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009')

In [56]:
rs_ = run_hpc_inference(
    model_path=MODEL_PATH,
    image_path=BAD_IM_PATH,
    output_dir=OUTPUT_DIR,
    save_heatmaps=True,
    heatmap_style="side_by_side",
    batch_size=10,
    num_nodes=10,
    compress=True,
    jpeg_quality=95,
    preprocessing_fn=None,
    preprocessing_kwargs=None,
)

🖥️  Running HPC multinode inference
🚀 SMART FOLDER INFERENCE DISTRIBUTION

📋 Step 1: Validating inputs...
✅ Model found: /home/ai_dsx.work/data/projects/AD_tool_test/models/exports/TEST_MULITNODE_task_000_padim_resnet18_18_layer1/weights/torch/model.pt
✅ Root path valid: /home/ai_dsx.work/data/projects/AD_tool_test/images/bad
✅ Batch size valid: 10
✅ Number of nodes valid: 10
✅ Output directory: /home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009

📡 Step 2: Scanning folder structure...
📄 Detected FLAT structure in: /home/ai_dsx.work/data/projects/AD_tool_test/images/bad
   📷 Total images: 2

✅ Scan complete: 2 total images

🔨 Step 3: Creating smart batches...

🏭 Step 4: Creating HPC jobs...

🎯 INFERENCE JOB DISTRIBUTION SUMMARY

📁 Data Structure:
   Type: FLAT
   Total Images: 2

📦 Batch Configuration:
   Total Batches: 1
   Batch Sizes: min=2, max=2, avg=2

🏭 HPC Jobs:
   Total Jobs Created: 1
   Jobs per Node (approx): 0.1

🖥️  Compute Resources:
   Number of Nodes

Total:   0%|                                             | 0/19 [00:00<?, ?it/s]
0it [00:00, ?it/s]














































RUNNING:2, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]

RUNNING:3, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]


RUNNING:4, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]



RUNNING:5, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]




RUNNING:6, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]





RUNNING:7, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]






RUNNING:8, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]







RUNNING:9, DONE:0:   0%|                                 | 0/19 [00:00<?, ?it/s]








RUNNING:10, DONE:0:   0%|                                | 0/19 [00:00<?, ?it/s]


RUNNING:10, DONE:0:   0%|                                | 0/19 [00:00<?, ?it/s]

RUNNING:10, DONE:0:   0%|                                | 0/19 [00:00<?, ?it/s]






RUNNING:10, DONE:0:


✅ Job execution completed!
✅ HPC inference complete: 2 images processed


### Test done 

1. run_jupyter_inference
2. distribute_folder_inference

### Strategy 3: Parallel Execution (Python Script)


In [73]:
#| export
def _process_batch_worker(args: tuple) -> Dict[str, Any]:
    """Worker function for parallel batch processing."""
    model_path, batch_images, batch_id, output_dir, save_heatmaps, heatmap_style, compress, jpeg_quality, preprocessing_fn, preprocessing_kwargs, kwargs = args
    
    # Create batch file
    batch_list_file = Path(output_dir) / "batch_lists" / f"{batch_id}_images.txt"
    create_batch_list_file(batch_images, batch_list_file)
    
    # Process batch
    results = predict_image_list_from_file(
        model_path=model_path,
        image_list_file=batch_list_file,
        batch_id=batch_id,
        output_dir=output_dir,
        save_results=save_heatmaps,
        heatmap_style=heatmap_style,
        compress=compress,
        jpeg_quality=jpeg_quality,
        preprocessing_fn=preprocessing_fn,
        preprocessing_kwargs=preprocessing_kwargs,
        **kwargs
    )
    
    return {
        "batch_id": batch_id,
        "num_images": len(batch_images),
        "results": results
    }


In [84]:
#| export
def run_parallel_inference(
    model_path: Union[str, Path],
    image_path: Union[str, Path, List[Path]],
    batch_size: int = 100,
    num_workers: Optional[int] = None,
    output_dir: Union[str, Path] = "parallel_results",
    save_heatmaps: bool = False,
    heatmap_style: str = "side_by_side",
    compress: bool = True,
    jpeg_quality: int = 95,
    num_nodes: int = 10,
    preprocessing_fn=None,
    preprocessing_kwargs=None,
    **kwargs
) -> Dict[str, Any]:
    """Execute inference in parallel using multiprocessing.Pool."""

    print("🚀 SMART FOLDER INFERENCE DISTRIBUTION")
    print("="*70)
    
    # Step 1: Validate inputs (fail fast)
    print("\n📋 Step 1: Validating inputs...")
    model_path, root_path = validate_inference_inputs(
        model_path=model_path,
        root_path=image_path,
        batch_size=batch_size,
        num_nodes=num_nodes
    )
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"✅ Output directory: {output_dir}")
    
    # Step 2: Scan folder structure (auto-detect flat vs nested)
    print(f"\n📡 Step 2: Scanning folder structure...")
    folder_info = scan_folder_structure(root_path)
    
    # Step 3: Create smart batches
    print(f"\n🔨 Step 3: Creating smart batches...")
    batches = create_smart_batches(folder_info, batch_size)



    # Split into batches
    image_batches = create_smart_batches(
        Path(image_path), batch_size=batch_size)
    
    if num_workers is None:
        num_workers = cpu_count()
    
    print(f"⚡ Running in parallel mode with {num_workers} workers")
    
    model_path = Path(model_path)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    
    # Prepare worker arguments
    worker_args = []
    for i, batch in enumerate(image_batches):
        batch_id = f"batch_{i+1:04d}"
        batch_kwargs = {
            "save_heatmaps": save_heatmaps,
            "heatmap_style": heatmap_style,
            "compress": compress,
            "jpeg_quality": jpeg_quality,
            "preprocessing_fn": preprocessing_fn,
            "preprocessing_kwargs": preprocessing_kwargs,
            **kwargs
        }
        worker_args.append((model_path, batch, batch_id, output_path, batch_kwargs))
    
    # Execute in parallel
    with Pool(processes=num_workers) as pool:
        batch_results = list(tqdm(
            pool.imap(_process_batch_worker, worker_args),
            total=len(worker_args),
            desc="Processing batches"
        ))
    
    print(f"✅ Parallel inference complete: {len(image_batches)} batches processed")
    
    return {
        "mode": "parallel",
        "total_images": len(image_paths),
        "num_batches": len(image_batches),
        "num_workers": num_workers,
        "output_dir": str(output_path),
        "results": batch_results
    }


In [85]:
rs_ = run_parallel_inference(
    model_path=MODEL_PATH,
    image_path=BAD_IM_PATH,
    output_dir=OUTPUT_DIR,
    save_heatmaps=True,
    heatmap_style="side_by_side",
    compress=True,
    jpeg_quality=95,
    preprocessing_fn=None,
    preprocessing_kwargs=None,
)

🚀 SMART FOLDER INFERENCE DISTRIBUTION

📋 Step 1: Validating inputs...
✅ Model found: /home/ai_dsx.work/data/projects/AD_tool_test/models/exports/TEST_MULITNODE_task_000_padim_resnet18_18_layer1/weights/torch/model.pt
✅ Root path valid: /home/ai_dsx.work/data/projects/AD_tool_test/images/bad
✅ Batch size valid: 100
✅ Number of nodes valid: 10
✅ Output directory: /home/ai_dsx.work/data/projects/AD_tool_test/inference_results20251009

📡 Step 2: Scanning folder structure...
📄 Detected FLAT structure in: /home/ai_dsx.work/data/projects/AD_tool_test/images/bad
   📷 Total images: 2

✅ Scan complete: 2 total images

🔨 Step 3: Creating smart batches...


TypeError: 'PosixPath' object is not subscriptable

## Unified Interface

The main function that intelligently routes to the appropriate execution strategy.


In [ ]:
#| export
def unified_inference(
    model_path: Union[str, Path],
    test_folders: Union[str, Path, List[Union[str, Path]]],
    batch_size: int = 100,
    execution_mode: str = "auto",
    num_nodes: int = 4,
    num_workers: Optional[int] = None,
    output_dir: Optional[Union[str, Path]] = None,
    save_heatmaps: bool = False,
    heatmap_style: str = "cv2_side_by_side",
    **kwargs
) -> Dict[str, Any]:
    """Unified inference function that automatically selects execution strategy.
    
    Args:
        model_path: Path to trained model
        test_folders: Image folder(s) or file path(s)
        batch_size: Maximum images per batch
        execution_mode: "auto", "jupyter", "hpc", or "parallel"
        num_nodes: Number of HPC nodes (for HPC mode)
        num_workers: Number of parallel workers (for parallel mode, default: cpu_count())
        output_dir: Output directory for results
        save_heatmaps: Whether to save visualization heatmaps
        heatmap_style: Visualization style
        **kwargs: Additional arguments passed to inference functions
    
    Returns:
        Dictionary with inference results and metadata
    """
    
    print("🚀 Starting Unified Inference System")
    print("=" * 50)
    
    # Auto-detect or validate execution mode
    if execution_mode == "auto":
        execution_mode = detect_execution_environment()
        print(f"🎯 Auto-detected mode: {execution_mode}")
    else:
        if execution_mode not in ["jupyter", "hpc", "parallel"]:
            raise ValueError(f"Invalid execution_mode: {execution_mode}")
        print(f"🎯 Using specified mode: {execution_mode}")
    
    # Resolve image paths (handles lists, flat folders, and nested folders)
    image_paths = resolve_test_folders_smart(test_folders)
    
    if not image_paths:
        raise ValueError("No valid images found in test_folders")
    
    # Set default output directory based on mode
    if output_dir is None:
        output_dir = f"{execution_mode}_inference_results"
    
    # Execute based on detected/specified mode
    print("=" * 50)
    
    if execution_mode == "jupyter":
        results = run_jupyter_inference(
            model_path=model_path,
            image_paths=image_paths,
            output_dir=output_dir,
            save_heatmaps=save_heatmaps,
            heatmap_style=heatmap_style,
            **kwargs
        )
    
    elif execution_mode == "hpc":
        results = run_hpc_multinode_inference(
            model_path=model_path,
            image_paths=image_paths,
            batch_size=batch_size,
            num_nodes=num_nodes,
            output_dir=output_dir,
            save_heatmaps=save_heatmaps,
            heatmap_style=heatmap_style,
            **kwargs
        )
    
    elif execution_mode == "parallel":
        results = run_parallel_inference(
            model_path=model_path,
            image_paths=image_paths,
            batch_size=batch_size,
            num_workers=num_workers,
            output_dir=output_dir,
            save_heatmaps=save_heatmaps,
            heatmap_style=heatmap_style,
            **kwargs
        )
    
    print("=" * 50)
    print(f"🎉 Unified inference complete!")
    print(f"   Mode: {results['mode']}")
    print(f"   Images: {results['total_images']}")
    print(f"   Output: {results['output_dir']}")
    
    return results


## Usage Examples


In [ ]:
# Example 1: Auto-detect and run with flat folder
# results = unified_inference(
#     model_path="path/to/model.pt",
#     test_folders="test_images/",  # Flat folder - all images in one directory
#     batch_size=50,
#     save_heatmaps=True
# )

# Example 1b: Auto-detect and run with nested folder
# results = unified_inference(
#     model_path="path/to/model.pt",
#     test_folders="production_images/",  # Nested folder - images in subfolders
#     batch_size=50,
#     save_heatmaps=True
# )

# Example 1c: Auto-detect and run with list of images
# results = unified_inference(
#     model_path="path/to/model.pt",
#     test_folders=["img1.jpg", "img2.png", "folder1/"],  # Mixed: files and folders
#     batch_size=50,
#     save_heatmaps=True
# )


In [ ]:
# Example 2: Force Jupyter mode
# results = unified_inference(
#     model_path="path/to/model.pt",
#     test_folders=["folder1/", "folder2/"],
#     execution_mode="jupyter"
# )


In [ ]:
# Example 3: Force HPC multinode mode
# results = unified_inference(
#     model_path="path/to/model.pt",
#     test_folders="large_dataset/",
#     execution_mode="hpc",
#     batch_size=100,
#     num_nodes=8
# )


In [ ]:
# Example 4: Force parallel mode with custom workers
# results = unified_inference(
#     model_path="path/to/model.pt",
#     test_folders="test_images/",
#     execution_mode="parallel",
#     batch_size=50,
#     num_workers=8
# )


## Tests and Export


In [ ]:
def test_unified_inference_modes():
    """Test that unified inference can route to different modes."""
    
    # Test mode validation
    try:
        unified_inference(
            model_path="fake.pt",
            test_folders=[],
            execution_mode="invalid_mode"
        )
        assert False, "Should raise ValueError for invalid mode"
    except ValueError as e:
        assert "Invalid execution_mode" in str(e)
        print("✅ Mode validation works")
    
    # Test empty image paths
    try:
        unified_inference(
            model_path="fake.pt",
            test_folders=[],
            execution_mode="jupyter"
        )
        assert False, "Should raise ValueError for empty images"
    except ValueError as e:
        assert "No valid images" in str(e)
        print("✅ Empty images validation works")
    
    print("✅ All unified inference tests passed")

def test_resolve_test_folders_smart():
    """Test smart folder resolution with different input types."""
    
    # Test with non-existent path (should warn but not fail)
    result = resolve_test_folders_smart(["non_existent_folder"])
    test_eq(len(result), 0)
    print("✅ Handles non-existent paths gracefully")
    
    # Test with empty list
    result = resolve_test_folders_smart([])
    test_eq(len(result), 0)
    print("✅ Handles empty list")
    
    print("✅ Smart folder resolution tests passed")

# Run tests
test_unified_inference_modes()
test_resolve_test_folders_smart()


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
